In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from utils import chart_price, plot, perf, v_backtester, c_backtester, perf_var, summary
import sys
sys.path.append('/home/tomek/ib_tools')
import matplotlib.pyplot as plt
%matplotlib inline
from tester import get_data
from numba import jit
from typing import Optional, Union, List
from indicators import get_ATR

In [2]:
from numba_tools import stop_loss, swing

In [3]:
df_master = get_data('NQ')
df = df_master.loc['20180901': '20190331']
df_out = df_master.loc['20190331': '20190930']
#df = df_master.resample('1H').apply({'open': 'first', 'high': 'max', 'low': 'min', 
#                                      'close': 'last', 'volume': 'sum', 'average': 'mean', 'barCount': 'sum'}).dropna()
df

,open,high,low,close,volume,average,barCount
date,,,,,,,
2018-09-03 00:00:00,7670.00,7675.00,7670.00,7671.50,166,7672.550,104
2018-09-03 00:01:00,7671.50,7672.75,7670.00,7670.00,84,7671.675,60
2018-09-03 00:02:00,7670.00,7671.00,7669.75,7671.00,28,7670.250,21
2018-09-03 00:03:00,7671.25,7672.25,7671.25,7671.75,45,7671.850,31
2018-09-03 00:04:00,7672.00,7672.00,7669.75,7669.75,32,7670.775,26
...,...,...,...,...,...,...,...
2019-03-29 21:55:00,7413.50,7413.75,7410.50,7410.75,102,7412.175,58
2019-03-29 21:56:00,7411.25,7411.50,7410.00,7410.50,71,7410.850,52
2019-03-29 21:57:00,7410.50,7412.75,7409.50,7412.75,71,7410.650,47


In [4]:
df['position'] = swing(df, 75, 1)[:,3]
df

== Pipeline: nopython for numba_tools._swing
== -- analyzing bytecode
== -- fix up args
== -- processing IR
== -- Handle with contexts
== -- rewrite semantic constants
== -- dead branch pruning
== -- nopython rewrites
== -- inline calls to locally defined closures
== -- convert make_function into JIT functions
== -- inline inlinable functions
== -- dead branch pruning
== -- find literally calls
== -- handles literal_unroll
== -- ssa
== -- nopython frontend
== Pipeline: nopython for numba.np.arrayobj.ol_bool.<locals>.impl
== -- analyzing bytecode
== -- fix up args
== -- processing IR
== -- Handle with contexts
== -- rewrite semantic constants
== -- dead branch pruning
== -- nopython rewrites
== -- inline calls to locally defined closures
== -- convert make_function into JIT functions
== -- inline inlinable functions
== -- dead branch pruning
== -- find literally calls
== -- handles literal_unroll
== -- ssa
== -- nopython frontend
== -- annotate types
== -- remove phis nodes
== -- inline

,open,high,low,close,volume,average,barCount,position
date,,,,,,,,
2018-09-03 00:00:00,7670.00,7675.00,7670.00,7671.50,166,7672.550,104,1.0
2018-09-03 00:01:00,7671.50,7672.75,7670.00,7670.00,84,7671.675,60,1.0
2018-09-03 00:02:00,7670.00,7671.00,7669.75,7671.00,28,7670.250,21,1.0
2018-09-03 00:03:00,7671.25,7672.25,7671.25,7671.75,45,7671.850,31,1.0
2018-09-03 00:04:00,7672.00,7672.00,7669.75,7669.75,32,7670.775,26,1.0
...,...,...,...,...,...,...,...,...
2019-03-29 21:55:00,7413.50,7413.75,7410.50,7410.75,102,7412.175,58,1.0
2019-03-29 21:56:00,7411.25,7411.50,7410.00,7410.50,71,7410.850,52,1.0
2019-03-29 21:57:00,7410.50,7412.75,7409.50,7412.75,71,7410.650,47,1.0


In [5]:
#%%timeit
#df['stop'] = stop_loss(df, 200)
df['adjusted_stop'] = stop_loss(df, 200, tp_multiple=3, adjust=('fixed', 2, 2))
#df

== Pipeline: nopython for <dynamic>.ctor
== -- analyzing bytecode
== -- fix up args
== -- processing IR
== -- Handle with contexts
== -- rewrite semantic constants
== -- dead branch pruning
== -- nopython rewrites
== -- inline calls to locally defined closures
== -- convert make_function into JIT functions
== -- inline inlinable functions
== -- dead branch pruning
== -- find literally calls
== -- handles literal_unroll
== -- ssa
== -- nopython frontend
== Pipeline: nopython for numba_tools.Stop.__init__
== -- analyzing bytecode
== -- fix up args
== -- processing IR
== -- Handle with contexts
== -- rewrite semantic constants
== -- dead branch pruning
== -- nopython rewrites
== -- inline calls to locally defined closures
== -- convert make_function into JIT functions
== -- inline inlinable functions
== -- dead branch pruning
== -- find literally calls
== -- handles literal_unroll
== -- ssa
== -- nopython frontend
== -- annotate types
== -- remove phis nodes
== -- inline overloaded functi

== -- convert make_function into JIT functions
== -- inline inlinable functions
== -- dead branch pruning
== -- find literally calls
== -- handles literal_unroll
== -- ssa
== -- nopython frontend
== -- annotate types
== -- remove phis nodes
== -- inline overloaded functions
== -- nopython rewrites
== -- ensure IR is legal prior to lowering
== -- nopython mode backend
== -- dump parfor diagnostics
== Pipeline: nopython for <dynamic>.accessor
== -- analyzing bytecode
== -- fix up args
== -- processing IR
== -- Handle with contexts
== -- rewrite semantic constants
== -- dead branch pruning
== -- nopython rewrites
== -- inline calls to locally defined closures
== -- convert make_function into JIT functions
== -- inline inlinable functions
== -- dead branch pruning
== -- find literally calls
== -- handles literal_unroll
== -- ssa
== -- nopython frontend
== -- annotate types
== -- remove phis nodes
== -- inline overloaded functions
== -- nopython rewrites
== -- ensure IR is legal prior to lo

== -- annotate types
== -- remove phis nodes
== -- inline overloaded functions
== -- nopython rewrites
== -- ensure IR is legal prior to lowering
== -- nopython mode backend
== Pipeline: nopython for <dynamic>.accessor
== -- analyzing bytecode
== -- fix up args
== -- processing IR
== -- Handle with contexts
== -- rewrite semantic constants
== -- dead branch pruning
== -- nopython rewrites
== -- inline calls to locally defined closures
== -- convert make_function into JIT functions
== -- inline inlinable functions
== -- dead branch pruning
== -- find literally calls
== -- handles literal_unroll
== -- ssa
== -- nopython frontend
== -- annotate types
== -- remove phis nodes
== -- inline overloaded functions
== -- nopython rewrites
== -- ensure IR is legal prior to lowering
== -- nopython mode backend
== -- dump parfor diagnostics
== Pipeline: nopython for <dynamic>.accessor
== -- analyzing bytecode
== -- fix up args
== -- processing IR
== -- Handle with contexts
== -- rewrite semantic con

== -- handles literal_unroll
== -- ssa
== -- nopython frontend
== Pipeline: nopython for numba_tools.Context.eval_for_close
== -- analyzing bytecode
== -- fix up args
== -- processing IR
== -- Handle with contexts
== -- rewrite semantic constants
== -- dead branch pruning
== -- nopython rewrites
== -- inline calls to locally defined closures
== -- convert make_function into JIT functions
== -- inline inlinable functions
== -- dead branch pruning
== -- find literally calls
== -- handles literal_unroll
== -- ssa
== -- nopython frontend
== Pipeline: nopython for numba_tools.Context.close_position
== -- analyzing bytecode
== -- fix up args
== -- processing IR
== -- Handle with contexts
== -- rewrite semantic constants
== -- dead branch pruning
== -- nopython rewrites
== -- inline calls to locally defined closures
== -- convert make_function into JIT functions
== -- inline inlinable functions
== -- dead branch pruning
== -- find literally calls
== -- handles literal_unroll
== -- ssa
== -- n

== -- handles literal_unroll
== -- ssa
== -- nopython frontend
== Pipeline: nopython for numba.cpython.unicode.unicode_eq.<locals>.eq_impl
== -- analyzing bytecode
== -- fix up args
== -- processing IR
== -- Handle with contexts
== -- rewrite semantic constants
== -- dead branch pruning
== -- nopython rewrites
== -- inline calls to locally defined closures
== -- convert make_function into JIT functions
== -- inline inlinable functions
== -- dead branch pruning
== -- find literally calls
== -- handles literal_unroll
== -- ssa
== -- nopython frontend
== Pipeline: nopython for numba.cpython.unicode.unicode_str.<locals>.<lambda>
== -- analyzing bytecode
== -- fix up args
== -- processing IR
== -- Handle with contexts
== -- rewrite semantic constants
== -- dead branch pruning
== -- nopython rewrites
== -- inline calls to locally defined closures
== -- convert make_function into JIT functions
== -- inline inlinable functions
== -- dead branch pruning
== -- find literally calls
== -- handles 

== -- rewrite semantic constants
== -- dead branch pruning
== -- nopython rewrites
== -- inline calls to locally defined closures
== -- convert make_function into JIT functions
== -- inline inlinable functions
== -- dead branch pruning
== -- find literally calls
== -- handles literal_unroll
== -- ssa
== -- nopython frontend
== Pipeline: nopython for numba_tools.BaseBracket.open
== -- analyzing bytecode
== -- fix up args
== -- processing IR
== -- Handle with contexts
== -- rewrite semantic constants
== -- dead branch pruning
== -- nopython rewrites
== -- inline calls to locally defined closures
== -- convert make_function into JIT functions
== -- inline inlinable functions
== -- dead branch pruning
== -- find literally calls
== -- handles literal_unroll
== -- ssa
== -- nopython frontend
== Pipeline: nopython for numba_tools.BaseBracket.set_entry
== -- analyzing bytecode
== -- fix up args
== -- processing IR
== -- Handle with contexts
== -- rewrite semantic constants
== -- dead branch pr

TypeError: expected instance.jitclass.Stop#7f09bff11130<distance:float32,position:int32,entry:float32,trigger:float32,mode:unicode_type>, got None

In [ ]:
df[df['stop'] != df['adjusted_stop']]

In [ ]:
df[['close', 'position', 'stop', 'adjusted_stop']
  ].plot(figsize=(20,10), secondary_y=['position', 'stop', 'adjusted_stop']);

In [ ]:
df[['close', 'position', 'adjusted_stop']].iloc[:16000].plot(figsize=(20,8), secondary_y=['position', 'adjusted_stop']);

In [ ]:
df[['close', 'position', 'stop']].iloc[:16000].plot(figsize=(20,8), secondary_y=['position', 'stop']);